In [10]:
!pip install --user --upgrade kfp google-cloud-aiplatform google-cloud-pipeline-components

  Obtaining dependency information for google-cloud-pipeline-components from https://files.pythonhosted.org/packages/df/e8/04e989b5e22d7cfc1208d15852e3ef57c881b439a4cfe241372d1fcdde77/google_cloud_pipeline_components-2.13.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 43.9 MB/s eta 0:00:00


In [9]:
import kfp
from kfp import compiler
from google.cloud import aiplatform
import os
from datetime import datetime

In [2]:
# The project and bucket are for experiments below.
PROJECT_ID = "donuts-dev"  # @param {type:"string"}

# The form for BUCKET_URI is gs://<bucket-name>.
BUCKET_URI = "gs://donuts-dev-usc1"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}
PIPELINE_ROOT_PATH = BUCKET_URI + "/pipeline_root"

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(STAGING_BUCKET, "keras_yolov8")

In [17]:

TRAIN_MACHINE_TYPE = "n1-highmem-16"
TRAIN_ACCELERATOR_TYPE = "NVIDIA_TESLA_V100"
TRAIN_NUM_GPU = 2
TRAIN_CONTAINER_URI = (
    "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/keras-yolov8-train"
)
TRAINING_JOB_PREFIX = "train_yolov8"
MODEL_DISPLAY_NAME = "yolov8_fine_tuned"

UPLOAD_JOB_PREFIX = "upload_yolov8"
DEPLOY_JOB_PREFIX = "deploy_yolov8"
SERVING_CONTAINER_URI = (
    "us-docker.pkg.dev/vertex-ai-restricted/prediction/tf_opt-gpu.2-12:latest"
)
SERVING_ACCELERATOR_TYPE = "NVIDIA_TESLA_T4"
SERVING_MACHINE_TYPE = "n1-standard-4"
SERVING_CONTAINER_ARGS = ["--allow_precompilation", "--allow_compression"]

RESOLUTION = 512

def get_job_name_with_datetime(prefix: str):
    """Generates a job name with date time when triggering training or deployment
    jobs in Vertex AI.
    """
    return prefix + datetime.now().strftime("_%Y%m%d_%H%M%S")

In [7]:
train_job_name = get_job_name_with_datetime(TRAINING_JOB_PREFIX)
model_dir = os.path.join(MODEL_BUCKET, train_job_name)
@kfp.dsl.pipeline(
    name="yolov8_finetuning"
)
def pipeline(
    project: str = PROJECT_ID,
    model_display_name: str = MODEL_DISPLAY_NAME,
    input_csv_path:str = "gs://cloud-samples-data/vision/salads.csv",
    epochs: int = 10,
    learning_rate: float = 0.0005,
    fpn_depth: int  = 3,
    confidence_threshold: float = 0.02,
    iou_threshold: float = 0.3,
    backbone:str = "yolo_v8_xl_backbone_coco",
    gpu_count: int = TRAIN_NUM_GPU
):
    
    worker_pool_specs = [
    {
        "machine_spec": {
            "machine_type": TRAIN_MACHINE_TYPE,
            "accelerator_type": TRAIN_ACCELERATOR_TYPE,
            "accelerator_count": TRAIN_NUM_GPU,
        },
        "replica_count": 1,
        "disk_spec": {
            "boot_disk_type": "pd-ssd",
            "boot_disk_size_gb": 500,
        },
        "container_spec": {
            "image_uri": TRAIN_CONTAINER_URI,
            "command": [],
            "env": [
                {
                    "name": "RESOLUTION",
                    "value": f"{RESOLUTION}",
                },
            ],
            "args": [
                f"--input_csv_path={input_csv_path}",
                f"--output_model_dir={model_dir}",
                f"--epochs={epochs}",
                f"--pretrained_backbone={backbone}",
                f"--fpn_depth={fpn_depth}",
                f"--learning_rate={learning_rate}",
                f"--confidence_threshold={confidence_threshold}",
                f"--iou_threshold={iou_threshold}",
            ],
        },
    }
    ]
    
    from google_cloud_pipeline_components.types import artifact_types
    from google_cloud_pipeline_components.v1.custom_job import \
        CustomTrainingJobOp
    from google_cloud_pipeline_components.v1.model import ModelUploadOp
    from kfp.dsl import importer_node

    fine_tuning_op = CustomTrainingJobOp(
        project=project,
        display_name=get_job_name_with_datetime(TRAINING_JOB_PREFIX),
        worker_pool_specs=worker_pool_specs
    )
    
    import_unmanaged_model_task = importer_node.importer(
        artifact_uri=model_dir,
        artifact_class=artifact_types.UnmanagedContainerModel,
        metadata={
            "containerSpec": {
                "imageUri": SERVING_CONTAINER_URI,
            },
        },
    ).after(fine_tuning_op)
    
    model_upload_op = ModelUploadOp(
        project=project,
        display_name=model_display_name,
        unmanaged_container_model=import_unmanaged_model_task.outputs["artifact"]
    )
    
    


In [10]:
compiler.Compiler().compile(
    pipeline_func=pipeline,
    package_path="yolov8_finetune_pipeline.json",
)

In [16]:
DISPLAY_NAME = get_job_name_with_datetime("yolov8_finetuning")

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="yolov8_finetune_pipeline.json",
    parameter_values={"epochs":1}
    pipeline_root=PIPELINE_ROOT_PATH,
    enable_caching=False,
)

job.run()

Creating PipelineJob
PipelineJob created. Resource name: projects/633265597134/locations/us-central1/pipelineJobs/yolov8-finetuning-20240424103919
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/633265597134/locations/us-central1/pipelineJobs/yolov8-finetuning-20240424103919')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/yolov8-finetuning-20240424103919?project=633265597134


RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [custom-training-job].; Job (project_id = donuts-dev, job_id = 1919405628858564608) is failed due to the above error.; Failed to handle the job: {project_number = 633265597134, job_id = 1919405628858564608}"
